#STEP-1) Install libtorrent

In [ ]:
!apt install python3-libtorrent

#STEP-2) Torrent-to-MP4

0.   Connect to your Google-Drive
1.   PASTE THE MAGENT LINK
2.   SELECT THE FILES YOU WANT TO DOWNLOAD IN THE TORRENT AND ENTER 'done'




In [ ]:
import libtorrent as lt
import time
from tqdm import tqdm
import os
import subprocess
import re
from google.colab import drive

# Function to convert seconds into hh:mm:ss format
def convert_to_hms(seconds):
    return time.strftime('%H:%M:%S', time.gmtime(seconds))

# Mount Google Drive
drive.mount('/content/drive')

# Install ffmpeg
!apt install -y ffmpeg

# Set save path
save_path = "/content/temp"
final_path = "/content/drive/My Drive/Torrent"

# Ask for magnet link
magnet_link = input("Please enter the magnet link: ")

# Create session and add magnet link
ses = lt.session()
params = {
    'save_path': save_path,
}
handle = lt.add_magnet_uri(ses, magnet_link, params)

# Wait for metadata to download
print("Downloading Metadata...")
while not handle.has_metadata():
    time.sleep(1)

# Get torrent info
torrent_info = handle.get_torrent_info()

# Display list of files and their sizes
print("Select the files you want to download:")
for i, file in enumerate(torrent_info.files()):
    print(f"{i+1}. {file.path} ({file.size / 1024 / 1024:.2f} MB)")

# Create list of selected files
selected_files = []
while True:
    selection = input("Enter file numbers to download (comma-separated), or 'done' to start downloading: ")
    if selection.lower() == 'done':
        break
    selected_files.extend(int(i) - 1 for i in selection.split(','))

# Set file priorities
for i in range(torrent_info.num_files()):
    handle.file_priority(i, 1 if i in selected_files else 0)

# Start downloading
print("Downloading...")
pbar = tqdm(total=100, ncols=70)
prev_progress = 0

while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    progress = s.progress * 100
    pbar.set_description("Progress: %s%% | State: %s | Seeds: %s | Peers: %s | dl: %.2f kB/s | ul: %.2f kB/s" % (
        int(progress),
        ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume'][s.state],
        s.num_seeds,
        s.num_peers,
        s.download_rate / 1000,
        s.upload_rate / 1000
    ))
    increment = progress - prev_progress
    pbar.update(increment)
    prev_progress = progress
    time.sleep(1)

pbar.close()
print("Download complete!")

# Convert video files to mp4
for i in selected_files:
    file = torrent_info.files().at(i)
    if file.path.endswith(('.mkv', '.avi', '.mov')):
        print(f"Converting {file.path} to mp4...")

        # Get video duration
        command = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', os.path.join(save_path, file.path)]
        duration = float(subprocess.check_output(command).decode('utf-8').strip())

        # Start FFmpeg conversion process
        command = ['ffmpeg', '-i', os.path.join(save_path, file.path), '-c:v', 'h264_nvenc', '-preset', 'slow', '-c:a', 'copy', '-c:s', 'mov_text', os.path.splitext(os.path.join(final_path, file.path))[0] + '.mp4']
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

        # Initialize progress bar
        pbar = tqdm(total=duration, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]')

        # Update progress bar based on FFmpeg output
        for line in process.stdout:
            match = re.search(r'time=(\d+:\d+:\d+\.\d+)', line)
            if match:
                time_str = match.group(1)
                hours, minutes, seconds = map(float, time_str.split(':'))
                time_seconds = hours*3600 + minutes*60 + seconds
                pbar.update(time_seconds - pbar.n)

        pbar.close()

        print(f"Conversion of {file.path} complete!")